# Conjugate Priors on Feature Dataset (Beta-Binomial Direction Model)

We reuse the Chapter 10 conjugate-prior idea, but source the returns from
`dataset/features/all_features.parquet` rather than computing them from cleaned OHLCV.

Model:
- $y_t=1$ if next-day return is positive, else $0$
- $y_t\sim\text{Bernoulli}(p)$, $p\sim\text{Beta}(\alpha_0,\beta_0)$
- Posterior mean gives $\hat p_t$; we trade on $\hat p_t-0.5$.


# Bayesian Models on Feature Dataset: Conjugate Priors (Feature Store)

These notebooks train on the exported feature store: `dataset/features/all_features.parquet`.

Backtesting is identical to the `ML_Linear_Models_*` notebooks: we build a daily
prediction matrix (Date x Asset_ID), convert it to weekly-rebalanced portfolios
(1/N and MPT), and run the vectorized backtest engine.

In [1]:
from __future__ import annotations

from pathlib import Path
import sys

import numpy as np
import pandas as pd

SEED = 42
rng = np.random.default_rng(SEED)

def find_project_root(start: Path) -> Path:
    p = start.resolve()
    for _ in range(10):
        if (p / 'src').exists() and (p / 'dataset').exists():
            return p
        p = p.parent
    raise RuntimeError(f'Could not find project root from: {start!s}')

PROJECT_ROOT = find_project_root(Path.cwd())
FEATURES_PARQUET = PROJECT_ROOT / 'dataset' / 'features' / 'all_features.parquet'

src_dir = PROJECT_ROOT / 'src'
if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))


In [2]:
TARGET_COL = 'ret_1d'
TARGET_FWD_COL = 'y_ret_1d_fwd'

df = pd.read_parquet(FEATURES_PARQUET)
if 'Date' in df.columns:
    df['Date'] = pd.to_datetime(df['Date'])
    df = df.set_index('Date')
df = df.sort_index().copy()
df[TARGET_FWD_COL] = df.groupby('Asset_ID', sort=False)[TARGET_COL].shift(-1)
df = df.dropna(subset=[TARGET_FWD_COL])
df['y_up_fwd'] = (df[TARGET_FWD_COL] > 0).astype(int)

# Time split (same policy)
TRAIN_YEARS = 7
VAL_MONTHS = 18
TEST_MONTHS = 18

def align_to_trading_date(index: pd.DatetimeIndex, ts: pd.Timestamp) -> pd.Timestamp:
    pos = int(index.searchsorted(ts, side='left'))
    if pos >= len(index):
        return pd.Timestamp(index[-1])
    return pd.Timestamp(index[pos])

idx = pd.DatetimeIndex(df.index.unique()).sort_values()
end = pd.Timestamp(idx[-1])
raw_test_start = end - pd.DateOffset(months=TEST_MONTHS)
raw_val_start = raw_test_start - pd.DateOffset(months=VAL_MONTHS)
raw_train_start = raw_val_start - pd.DateOffset(years=TRAIN_YEARS)
test_start = align_to_trading_date(idx, pd.Timestamp(raw_test_start))
val_start = align_to_trading_date(idx, pd.Timestamp(raw_val_start))
train_start = align_to_trading_date(idx, pd.Timestamp(raw_train_start))
print('aligned boundaries:', train_start, val_start, test_start, end)

y_mat = df.pivot_table(index=df.index, columns='Asset_ID', values='y_up_fwd', aggfunc='mean').sort_index()

ALPHA0 = 1.0
BETA0 = 1.0
LOOKBACK_DAYS = 252

s = y_mat.rolling(LOOKBACK_DAYS, min_periods=10).sum().shift(1)
n = y_mat.rolling(LOOKBACK_DAYS, min_periods=10).count().shift(1)
p_up = (ALPHA0 + s) / (ALPHA0 + BETA0 + n)

pred_matrix = (p_up - 0.5).loc[test_start:end].fillna(0.0)


aligned boundaries: 2016-01-25 00:00:00 2023-01-17 00:00:00 2024-07-15 00:00:00 2026-01-15 00:00:00


In [3]:
from IPython.display import display
from bokeh.io import output_notebook, show

from backtester.data import load_cleaned_assets, align_close_prices
from backtester.engine import BacktestConfig, run_backtest
from backtester.report import compute_backtest_report
from backtester.bokeh_plots import build_interactive_portfolio_layout
from backtester.portfolio import equal_weight, optimize_mpt

output_notebook()

if 'pred_matrix' not in globals():
    raise RuntimeError('Expected `pred_matrix` (index=date, columns=Asset_ID) to exist')

pred_range = pd.DatetimeIndex(pred_matrix.index).sort_values()
if pred_range.empty:
    raise RuntimeError('pred_matrix has empty index')
bt_start = pd.Timestamp(pred_range[0])
bt_end = pd.Timestamp(pred_range[-1])

bt_assets = sorted([str(c) for c in pred_matrix.columns.tolist()])
CLEANED_DIR = PROJECT_ROOT / 'dataset' / 'cleaned'
assets_ohlcv = load_cleaned_assets(symbols=bt_assets, cleaned_dir=str(CLEANED_DIR))
close_prices = align_close_prices(assets_ohlcv)

pred_matrix = pred_matrix.reindex(close_prices.index)
close_prices = close_prices.loc[bt_start:bt_end]
pred_matrix = pred_matrix.loc[bt_start:bt_end]
returns_matrix = close_prices.pct_change().fillna(0.0)

market_df = pd.DataFrame({
    'Open': pd.concat([df['Open'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'High': pd.concat([df['High'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Low': pd.concat([df['Low'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Close': pd.concat([df['Close'] for df in assets_ohlcv.values()], axis=1).mean(axis=1),
    'Volume': pd.concat([df['Volume'] for df in assets_ohlcv.values()], axis=1).sum(axis=1),
}).sort_index().loc[bt_start:bt_end]

REBALANCE_FREQ = 'W'
TOP_K = min(20, len(bt_assets))
LOOKBACK_DAYS = 126

def build_weights_from_predictions(pred_matrix: pd.DataFrame, *, pm_style: str) -> pd.DataFrame:
    rebal_dates = set(pd.Series(pred_matrix.index, index=pred_matrix.index).resample(REBALANCE_FREQ).last().dropna().tolist())
    w_last = pd.Series(0.0, index=bt_assets)
    rows = []
    for dt in pred_matrix.index:
        if dt in rebal_dates:
            row = pred_matrix.loc[dt].dropna().sort_values(ascending=False)
            top = row.head(TOP_K)
            candidates = [a for a, v in top.items() if np.isfinite(v) and float(v) > 0.0]
            if not candidates:
                w_last = pd.Series(0.0, index=bt_assets)
            else:
                if pm_style == '1N':
                    w_dict = equal_weight(candidates)
                elif pm_style == 'MPT':
                    w_dict = optimize_mpt(returns_matrix, candidates, dt, lookback_days=LOOKBACK_DAYS)
                else:
                    raise ValueError(f'Unknown pm_style: {pm_style!r}')
                w_last = pd.Series(0.0, index=bt_assets)
                for a, w in w_dict.items():
                    w_last[str(a)] = float(w)
        rows.append(w_last)
    return pd.DataFrame(rows, index=pred_matrix.index, columns=bt_assets).fillna(0.0)

cfg = BacktestConfig(initial_equity=1_000_000.0, transaction_cost_bps=5.0, mode='vectorized')

compare_rows = []
results = {}
for pm_style in ['1N', 'MPT']:
    w = build_weights_from_predictions(pred_matrix, pm_style=pm_style)
    res = run_backtest(close_prices, w, config=cfg)
    rpt = compute_backtest_report(result=res, close_prices=close_prices)
    results[pm_style] = (w, res, rpt)
    compare_rows.append({
        'style': pm_style,
        'Total Return [%]': float(rpt['Total Return [%]']),
        'CAGR [%]': float(rpt['CAGR [%]']),
        'Sharpe': float(rpt['Sharpe']),
        'Max Drawdown [%]': float(rpt['Max Drawdown [%]']),
    })
compare = pd.DataFrame(compare_rows).sort_values('Total Return [%]', ascending=False).reset_index(drop=True)
display(compare)

BASE_TITLE = 'Conjugate Priors (Feature Store)'
for pm_style in ['1N', 'MPT']:
    w, res, rpt = results[pm_style]
    title = BASE_TITLE + ' - ' + pm_style
    display(rpt.to_frame(title))
    layout = build_interactive_portfolio_layout(
        market_ohlcv=market_df,
        equity=res.equity,
        returns=res.returns,
        weights=res.weights,
        turnover=res.turnover,
        costs=res.costs,
        close_prices=close_prices,
        title=title,
    )
    show(layout)


Loading BokehJS ...

,style,Total Return [%],CAGR [%],Sharpe,Max Drawdown [%]
0,1N,33.340227,21.145548,1.433001,-12.921751
1,MPT,13.645075,8.901471,0.637857,-13.418566


,Conjugate Priors (Feature Store) - 1N
Start,2024-07-15 00:00:00
End,2026-01-15 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1333402.271843
Equity Peak,1333402.271843
Total Return [%],33.340227
CAGR [%],21.145548
Volatility (ann) [%],14.04348
Sharpe,1.433001


,Conjugate Priors (Feature Store) - MPT
Start,2024-07-15 00:00:00
End,2026-01-15 00:00:00
Duration,549 days 00:00:00
Initial Equity,1000000.0
Final Equity,1136450.746691
Equity Peak,1136450.746691
Total Return [%],13.645075
CAGR [%],8.901471
Volatility (ann) [%],15.139313
Sharpe,0.637857
